<a href="https://colab.research.google.com/github/IR-123/Encampment_Map.github.io/blob/main/Gaza_Solidarity_Encampment_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Necessary Packages

In [76]:
import folium
from folium import IFrame
# import geopandas
import gspread
import gspread_dataframe as gdf
from oauth2client.client import GoogleCredentials
from google.colab import drive, auth
import pandas as pd
import base64
import re
import os
# import imageio
import cv2
import glob
import numpy
# import time
from google.auth import default
# from geopy.geocoders import Nominatim

## Authorize Colab Access to Google Drive

In [2]:
drive.mount('/content/drive')

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)


Mounted at /content/drive


## Encampment Data

In [67]:
worksheet = gc.open('Encampments').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_records()

# Convert to a DataFrame and render.
encampment_data = pd.DataFrame(rows)
encampment_data = encampment_data.convert_dtypes()
encampment_data.dtypes

encampment_data['Latitude'] = pd.to_numeric(encampment_data['Latitude'], errors='coerce')
encampment_data['Longitude'] = pd.to_numeric(encampment_data['Longitude'], errors='coerce')
# encampment_data['Encampment Start Date'] = pd.to_datetime(encampment_data['Encampment Start Date'], errors='coerce')
# encampment_data['Date of raid/arrests'] = pd.to_datetime(encampment_data['Date of raid/arrests'], errors='coerce')
encampment_data['Location'] = encampment_data['City'].str.strip() + ", " + encampment_data['State'].str.strip()

encampment_data.head()

,University Name,City,State,Country,Latitude,Longitude,Photo Location,Category,Status,Encampment Start Date,Police Violence Status,Image Source,Date of raid/arrests,Number of Arrests,Police Violence,Video_1,Video_2,Source,Location
0,Columbia University & Barnard College,New York,New York,U.S.,40.807384,-73.963036,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,04/17/2024,Mass arrests,PYM NYC,4/18/2024,114+,https://video.twimg.com/ext_tw_video/178101696...,,,https://www.instagram.com/reel/C53zTS0uLxg/?ut...,"New York, New York"
1,The New School,New York,New York,U.S.,40.734677,-73.991509,https://drive.google.com/drive/folders/12-V7pk...,Encampment,Encampment Ongoing,04/21/2024,Police violence,TNS SJP,NA,Alumni forcibly removed,,,,https://www.instagram.com/reel/C6JjZykLzGB/?ut...,"New York, New York"
2,New York University (NYU),New York,New York,U.S.,40.729675,-73.996925,https://drive.google.com/drive/folders/12-V7pk...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",04/22/2024,Mass arrests,National SJP,4/22/2024,115+,https://x.com/AshAgony/status/1782946121393811475,,,https://www.instagram.com/p/C6DrwzRuxhV/?utm_s...,"New York, New York"
3,Yale University,New Haven,Connecticut,U.S.,41.316307,-72.922585,https://drive.google.com/drive/folders/1x071zz...,Encampment,Encampment Dispersed,04/20/2024,Mass arrests,Yalies4palestine,4/22/2024,47+,,,,https://www.instagram.com/p/C6EF_mnL9k0/,"New Haven, Connecticut"
4,University of Minnesotta,Twin Cities,Minnesotta,U.S.,44.974000,-93.227700,https://drive.google.com/drive/folders/1U_tKJC...,Encampment,Encampment Ongoing,04/23/0204,Mass arrests,SJP UMN,,9+,,,,https://www.motherjones.com/politics/2024/04/g...,"Twin Cities, Minnesotta"


## Geolocation Data
Testing to create code that takes university, city, state/country and outputs coodrinates. Issue is some university names have special characters or a combo of two universities (e.g. Columbia University & Barnard College)

In [ ]:
# geolocator = Nominatim(user_agent = 'university-locator')

# updated_list = []
# for row in encampment_data['University Name']:
#     # address = row['University Name'].str.strip()
#     geo_loc = geolocator.geocode(row, timeout=None)
#     if geo_loc is not None:
#         coordinates = {}
#         coordinates['University Name'] = row
#         coordinates['Latitude'] = geo_loc.latitude
#         coordinates['Longitude'] = geo_loc.longitude
#         updated_list.append(coordinates)
# updated_list


## Creates columns for path to photos for each college/university

In [77]:
directory_path = '/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/'
directory_files = os.listdir(directory_path)

file_list = []
for i in encampment_data['University Name']:
  list_of_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Media/' + str(i) + '/*') # * means all if need specific format then *.jpg
  if list_of_files:
        latest_file = max(list_of_files, key=os.path.getctime)
  else:
      latest_file = None  # or use a default value for latest_file
  names = {}
  names['University Name'] = i
  names['Image Path'] = latest_file
  file_list.append(names)

file_df = pd.DataFrame(file_list)

encampment_data2 = pd.merge(encampment_data, file_df, on = 'University Name')
encampment_data2 = encampment_data2.dropna(subset=['Latitude', 'Longitude']) # removes rows missing coordinates. else there will be an error
encampment_data2.head()

## Eventually I'm trying to create code that loops through media files and searches for jpgs. Then compiles jpgs to a gif which will be added to the popup marker
# for imageName in encampment_data['University Name']:
#     frames = []
#     imageName = imageName.split(',')
#     for i in imageName:
#         frames.append(directory_path + str(i) + '/' + '*.jpeg')
#     with imageio.get_writer(directory_path + str(i) + '/' + '*.gif', mode='I', duration=2) as writer:
#         for filename in frames:
#             image = cv2.resize(imageio.imread(filename), (width, height))
#             writer.append_data(image)
#             writer.append_data(image)

,University Name,City,State,Country,Latitude,Longitude,Photo Location,Category,Status,Encampment Start Date,Police Violence Status,Image Source,Date of raid/arrests,Number of Arrests,Police Violence,Video_1,Video_2,Source,Location,Image Path
0,Columbia University & Barnard College,New York,New York,U.S.,40.807384,-73.963036,https://drive.google.com/drive/folders/1uJeIOa...,Encampment,Encampment Ongoing,04/17/2024,Mass arrests,PYM NYC,4/18/2024,114+,https://video.twimg.com/ext_tw_video/178101696...,,,https://www.instagram.com/reel/C53zTS0uLxg/?ut...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
1,The New School,New York,New York,U.S.,40.734677,-73.991509,https://drive.google.com/drive/folders/12-V7pk...,Encampment,Encampment Ongoing,04/21/2024,Police violence,TNS SJP,NA,Alumni forcibly removed,,,,https://www.instagram.com/reel/C6JjZykLzGB/?ut...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
2,New York University (NYU),New York,New York,U.S.,40.729675,-73.996925,https://drive.google.com/drive/folders/12-V7pk...,Encampment,"Encampment Dispersed, Solidarity Actions Continue",04/22/2024,Mass arrests,National SJP,4/22/2024,115+,https://x.com/AshAgony/status/1782946121393811475,,,https://www.instagram.com/p/C6DrwzRuxhV/?utm_s...,"New York, New York",/content/drive/MyDrive/Colab Notebooks/Gaza So...
3,Yale University,New Haven,Connecticut,U.S.,41.316307,-72.922585,https://drive.google.com/drive/folders/1x071zz...,Encampment,Encampment Dispersed,04/20/2024,Mass arrests,Yalies4palestine,4/22/2024,47+,,,,https://www.instagram.com/p/C6EF_mnL9k0/,"New Haven, Connecticut",/content/drive/MyDrive/Colab Notebooks/Gaza So...
4,University of Minnesotta,Twin Cities,Minnesotta,U.S.,44.974000,-93.227700,https://drive.google.com/drive/folders/1U_tKJC...,Encampment,Encampment Ongoing,04/23/0204,Mass arrests,SJP UMN,,9+,,,,https://www.motherjones.com/politics/2024/04/g...,"Twin Cities, Minnesotta",/content/drive/MyDrive/Colab Notebooks/Gaza So...


## Map and Markers

In [71]:
usa_coord = [37.0902, -95.7129] # Latitude, Longitude coord in decimal degrees

# Get the maximum and minimum latitude and longitude
max_lat, min_lat = encampment_data2['Latitude'].max(), encampment_data2['Latitude'].min()
max_lon, min_lon = encampment_data2['Longitude'].max(), encampment_data2['Longitude'].min()

# Create USA map with folium wrapper around leaflet.js
usa_map = folium.Map(location=usa_coord, zoom_start=4.4,
                     max_zoom=12, min_lat=min_lat,
                     max_lat = max_lat, min_lon=min_lon,
                     max_lon=max_lon)
folium.TileLayer('cartodbpositron').add_to(usa_map)

# Encampment Markers
## Map Icon
pal_flag = "/content/drive/My Drive/673813_map_map marker_national_country_palestinian_pin_flag_palestine.png"
encampment_icon = "/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Graphics and Icons/Tent.png"
icon = folium.CustomIcon(pal_flag, icon_size=(60, 60))  # Adjust icon size as needed , icon_size=(50, 50)

## Map Loop
for index, row in encampment_data2.iterrows():

  if row['Image Path'] is not None:
    with open(row['Image Path'], "rb") as img_file:
      encoded_image = base64.b64encode(img_file.read()).decode('utf-8')
  else:
    print(f"Warning: Image Path is None for row {index}")
    with open('/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/Pal_Flag_Missing_Photos_Msg.jpg', "rb") as img_file:
      encoded_image = base64.b64encode(img_file.read()).decode('utf-8')

  popup_html = f"""
        <!DOCTYPE html>
        <html>
        <h4 align="left" style="font-family:Calibri; color:red"><strong><u>{row['University Name']}</u><strong>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
        <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet"/>
        </h4>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa fa-map-marker"></i> Location: </strong> {row['Location']}
        </h5>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-regular fa-calendar-days"></i> Start Date:</strong> {row['Encampment Start Date']}
        </h5>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-solid fa-layer-group"></i> Status:</strong> {row['Status']}
        </h5>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-solid fa-layer-group"></i> Category: </strong> {row['Category']}
        </h5>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-solid fa-handcuffs"></i> Police Violence Status:</strong> {row['Police Violence Status']}
        </h5>
        <h5 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-solid fa-handcuffs"></i> Number of Arrests:</strong> {row['Number of Arrests']}
        </h5>
        <img src="data:image/jpeg;base64,{encoded_image}" alt="Image" style="width:250px;height:200px;">

        """
        # Trying to add code that looks at the video columns to see if there is a link. If yes, then add html line for link. Else, string saying "No Videos" or blank
        # if pd.notna(row['Police Violence']):
        #   f"""
        #   <h7 align="left" style="font-family:Calibri; color:green"> <strong><i class="fa-solid fa-video"></i> Videos:</strong>
        #     <a href={row['Police Violence']} target="_blank">Police Violence
        #     </a>
        #     <a href={row['Video_1']} target="_blank">Solidarity Actions Video 1
        #   </a>
        #   <a href={row['Video_2']} target="_blank">Solidarity Actions Video 2
        #   </a>
        # </h7>
        # """

        # <a href={row['Police Violence']}>'Police Violence'
        # </a>
        # <a href={row['Video_1']}>'Solidarity Actions Video 1'
        # </a>
        # <a href={row['Video_2']}>'Solidarity Actions Video 2'
        # </a>
  icon = folium.CustomIcon(encampment_icon, icon_size=(60, 60))  # Adjust icon size as needed , icon_size=(50, 50)
  popup = folium.Popup(popup_html, max_width=300)
  folium.vector_layers.Marker(location=[row['Latitude'],row['Longitude']],
                                popup = popup,
                                icon = icon,
                                tooltip=str(row['University Name'])).add_to(usa_map)

# Set the map's boundaries based on the maximum and minimum latitudes and longitudes
usa_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]], padding = (10,10))

# Define maximum and minimum bounds for the map
max_bounds = usa_map.get_bounds()
usa_map.max_bounds = max_bounds


In [ ]:
usa_map.save("/content/drive/MyDrive/Colab Notebooks/Gaza Solidarity Encampments/encampments_map.html")